<a href="https://colab.research.google.com/github/mleshen/pungen/blob/master/pungen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Untar BookCorpus

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!tar -xvf /content/drive/My\ Drive/Thesis/bookcorpus.tar.bz2


books_large_p2.txt
books_large_p1.txt


In [4]:
!head -20 books_large_p2.txt

usually , he would be tearing around the living room , playing with his toys .
but just one look at a minion sent him practically catatonic .
that had been megan 's plan when she got him dressed earlier .
he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older .
she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age .
`` are n't you being a good boy ? ''
she said .
mason barely acknowledged her .
instead , his baby blues remained focused on the television .
since the movie was almost over , megan knew she better slip into the bedroom and finish getting ready .
each time she looked into mason 's face , she was grateful that he looked nothing like his father .
his platinum blond hair and blue eyes were completely hers .
it was only his build that he was taking after his father .
where 

In [5]:
!ls -sh books_large_p2.txt

2.0G books_large_p2.txt


In [6]:
!ls -sh books_large_p1.txt

2.4G books_large_p1.txt


# Preprocess raw text

In [7]:
!pip install unidecode

     |████████████████████████████████| 245kB 4.8MB/s 


In [8]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [9]:
!python -m spacy validate

✔ Loaded compatibility table

====================== Installed models (spaCy v2.1.9) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.1.0   ✔
link      en               en_core_web_sm   2.1.0   ✔



In [10]:
!pip install ipdb

  Created wheel for ipdb: filename=ipdb-0.12.3-cp36-none-any.whl size=9223 sha256=fc4503d089beeef99422668ab1dabcf639bcd50b7d5416346da44a28ab4dc0a7
  Stored in directory: /root/.cache/pip/wheels/57/43/c5/614153606de8f5e358e266723f53254e70752f4ffc8c85ec63
Successfully built ipdb


In [11]:
%%writefile preprocess_raw_text.py
import argparse
import spacy
import re
from unidecode import unidecode
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
sentencizer = nlp.create_pipe('sentencizer')
nlp.add_pipe(sentencizer, first=True)

print(nlp.pipe_names)
print(nlp.pipeline)

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--input')
    parser.add_argument('--output')
    parser.add_argument('--min-len', type=int, default=5, help='minimum sentence length')
    parser.add_argument('--max-len', type=int, default=30, help='maximum sentence length')
    args = parser.parse_args()
    return args

def sentence_iter(file_):
    with open(file_, 'r', errors='ignore') as fin:
        cache = []
        for line in fin:
            line = re.sub('\s+', ' ', unidecode(line.strip()))
            cache.append(line)
            if len(cache) == 200:
                docs = nlp.pipe(cache)
                for doc in docs:
                    for s in doc.sents:
                        yield s
                cache = []
        if cache:
            docs = nlp.pipe(cache)
            for doc in docs:
                for s in doc.sents:
                    yield s

def main(args):
    count = 0
    print ("hello")
    #lines = sum(1 for _ in sentence_iter(args.input))
    #print (lines)
    with open(args.output, 'w') as fout, open(args.input, 'r') as fin:
        for s in tqdm(sentence_iter(args.input), total=100000000000):
            # s = line.strip().split()
            if len(s) >= args.min_len and len(s) <= args.max_len:
                l = ['{}|{}'.format(token.text, token.pos_) for token in s]
                fout.write(' '.join(l) + '\n')
                count += 1
    print (count)

if __name__ == '__main__':
    args = parse_args()
    main(args)


Writing preprocess_raw_text.py




---



In [12]:
!python preprocess_raw_text.py --input books_large_p1.txt --output outp_p1.txt

['sentencizer', 'tagger']
[('sentencizer', <spacy.pipeline.pipes.Sentencizer object at 0x7f5adf434550>), ('tagger', <spacy.pipeline.pipes.Tagger object at 0x7f5adf1e4128>)]
hello
  0% 40274446/100000000000 [7:27:42<18519:42:52, 1499.30it/s]
34299307


In [13]:
!head -20 /content/drive/My\ Drive/Thesis/outp_p1.txt


the|DET half|ADV -|PUNCT ling|VERB book|NOUN one|NUM in|ADP the|DET fall|NOUN of|ADP igneeria|NOUN series|NOUN kaylee|VERB soderburg|NOUN copyright|NOUN 2013|NUM kaylee|VERB soderburg|NOUN all|DET rights|NOUN reserved|VERB .|PUNCT
i|PRON wish|VERB i|PRON had|VERB a|DET better|ADJ answer|NOUN to|ADP that|DET question|NOUN .|PUNCT
starlings|NOUN ,|PUNCT new|ADJ york|NOUN is|VERB not|ADV the|DET place|NOUN you|PRON d|AUX expect|VERB much|ADJ to|PART happen|VERB .|PUNCT
its|DET a|DET small|ADJ quiet|ADJ town|NOUN ,|PUNCT the|DET kind|NOUN where|ADV everyone|NOUN knows|VERB your|DET name|NOUN .|PUNCT
its|DET a|DET place|NOUN where|ADV your|DET parents|NOUN would|AUX nt|ADV even|ADV care|VERB if|ADP you|PRON stayed|VERB out|PART late|ADJ biking|NOUN with|ADP your|DET friends|NOUN .|PUNCT
only|ADV because|ADP everyone|NOUN felt|VERB so|ADV safe|ADJ ,|PUNCT so|ADV comfy|ADJ .|PUNCT
they|PRON do|VERB nt|ADV know|VERB the|DET half|NOUN of|ADP it|PRON .|PUNCT
i|PRON know|VERB it|PRON all|DET and|

In [14]:
!head -20 /content/drive/My\ Drive/Thesis/outp_books_large.txt

usually|ADV ,|PUNCT he|PRON would|VERB be|VERB tearing|VERB around|ADP the|DET living|NOUN room|NOUN ,|PUNCT playing|VERB with|ADP his|DET toys|NOUN .|PUNCT
but|CCONJ just|ADV one|NUM look|NOUN at|ADP a|DET minion|NOUN sent|VERB him|PRON practically|ADV catatonic|ADJ .|PUNCT
that|DET had|VERB been|VERB megan|NOUN 's|PART plan|NOUN when|ADV she|PRON got|VERB him|PRON dressed|VERB earlier|ADV .|PUNCT
she|PRON liked|VERB to|PART think|VERB being|VERB surrounded|VERB by|ADP adults|NOUN and|CCONJ older|ADJ kids|NOUN was|VERB one|NUM reason|NOUN why|ADV he|PRON was|VERB a|DET such|ADJ a|DET good|ADJ talker|NOUN for|ADP his|DET age|NOUN .|PUNCT
`|PUNCT `|PUNCT are|VERB n't|ADV you|PRON being|VERB a|DET good|ADJ boy|NOUN ?|PUNCT ''|PUNCT
mason|NOUN barely|ADV acknowledged|VERB her|PRON .|PUNCT
instead|ADV ,|PUNCT his|DET baby|NOUN blues|NOUN remained|VERB focused|ADJ on|ADP the|DET television|NOUN .|PUNCT
since|ADP the|DET movie|NOUN was|VERB almost|ADV over|ADV ,|PUNCT megan|NOUN knew|VERB sh

# Preprocess BookCorpus

In [0]:
!pip install fairseq

In [0]:
%%writefile preprocess_bookcorpus.py # -*- coding: utf-8 -*-

import os
import codecs
import pickle
import argparse
import numpy as np

from fairseq.data.dictionary import Dictionary

from pungen.utils import sentence_iterator


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-dir', type=str, help="data directory path to save vocab and processed data")
    parser.add_argument('--vocab', type=str, help="corpus path for building vocab")
    parser.add_argument('--corpus', type=str, help="corpus path")
    parser.add_argument('--max-dist', type=int, default=5, help="maximum distance to the word")
    parser.add_argument('--max-vocab', type=int, default=-1, help="maximum number of vocab")
    parser.add_argument('--threshold', type=int, default=-1)
    parser.add_argument('--min-dist', type=int, default=0, help="minimum distance to the word")
    args = parser.parse_args()
    return args


class Preprocess(object):

    def __init__(self, max_dist=5, min_dist=0, data_dir='./data/'):
        self.max_dist = max_dist
        self.min_dist = min_dist
        self.data_dir = data_dir

    def get_lemma(self, w):
        # w: (token, lemma, tag)
        token, lemma, tag = w
        return token if lemma == '-PRON-' else lemma

    def skipgram(self, sentence, i):
        iword = sentence[i]
        left = sentence[max(i - self.max_dist, 0) : max(i - self.min_dist, 0)]
        right = sentence[i + 1 + self.min_dist : i + 1 + self.max_dist]
        n = self.max_dist - self.min_dist
        return iword, [self.unk for _ in range(n - len(left))] + left + right + [self.unk for _ in range(n - len(right))]

    def build(self, filepath=None, vocab_path=None, threshold=-1, max_vocab=-1):
        if vocab_path and os.path.exists(vocab_path):
            print("loading vocab from {}".format(vocab_path))
            d = Dictionary.load(vocab_path)
            print('vocab size {}'.format(len(d)))
        else:
            print("building vocab...")
            d = Dictionary()
            for step, line in enumerate(sentence_iterator(filepath)):
                if not step % 1000:
                    print("working on {}kth line".format(step // 1000), end='\r')
                tokens = [self.get_lemma(w) for w in line]
                for tok in tokens:
                    d.add_symbol(tok)
            d.finalize(threshold=threshold, nwords=max_vocab)
            print('build done. vocab size {}'.format(len(d)))
            d.save('{}/dict.txt'.format(self.data_dir))

        self.vocab = d
        self.unk = self.vocab.unk()

    def convert(self, filepath):
        print("converting corpus...")
        step = 0
        fout = open('{}/train.bin'.format(args.data_dir), 'wb')
        for step, line in enumerate(sentence_iterator(filepath)):
            if not step % 1000:
                print("working on {}kth line".format(step // 1000), end='\r')
            tokens = [self.get_lemma(w) for w in line]
            sent = [self.vocab.index(w) for w in tokens]
            if len(sent) <= (self.max_dist - self.min_dist + 1):
                continue
            for i in range(len(sent)):
                iword, owords = self.skipgram(sent, i)
                a = np.array([iword] + owords, dtype=np.uint16)
                fout.write(a.tobytes())
        fout.close()
        print("conversion done")


if __name__ == '__main__':
    args = parse_args()
    preprocess = Preprocess(max_dist=args.max_dist, min_dist=args.min_dist, data_dir=args.data_dir)
    preprocess.build(vocab_path=args.vocab, filepath=args.corpus, threshold=args.threshold, max_vocab=args.max_vocab)
    preprocess.convert(args.corpus)


# Train skip-gram model

In [0]:
%%writefile train_bookcorpus.py
# -*- coding: utf-8 -*-

import os
import pickle
import random
import argparse
import torch as t
import numpy as np

from tqdm import tqdm
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from fairseq.data.dictionary import Dictionary

from .model import Word2Vec, SGNS


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--name', type=str, default='sgns', help="model name")
    parser.add_argument('--data', type=str, default='./data/', help="data directory path")
    parser.add_argument('--vocab', type=str, default='./data/', help="data directory path")
    parser.add_argument('--save_dir', type=str, default='./pts/', help="model directory path")
    parser.add_argument('--e_dim', type=int, default=300, help="embedding dimension")
    parser.add_argument('--n_negs', type=int, default=20, help="number of negative samples")
    parser.add_argument('--epoch', type=int, default=100, help="number of epochs")
    parser.add_argument('--mb', type=int, default=4096, help="mini-batch size")
    parser.add_argument('--ss_t', type=float, default=1e-5, help="subsample threshold")
    parser.add_argument('--conti', action='store_true', help="continue learning")
    parser.add_argument('--weights', action='store_true', help="use weights for negative sampling")
    parser.add_argument('--cuda', action='store_true', help="use CUDA")
    return parser.parse_args()


class PermutedSubsampledCorpus(Dataset):

    def __init__(self, datapath, ws=None, window=5):
        self.window = window
        if ws is not None:
            self.data = []
            for iword, owords in self.read_data(datapath):
                if random.random() > ws[iword]:
                    self.data.append((iword, owords))
        else:
            self.data = [(iword, owords) for iword, owords in self.read_data(datapath)]

    def read_data(self, datapath):
        n = 2 * self.window + 1
        with open(datapath, 'rb') as fin:
            print('Reading binary data...')
            data = np.fromfile(fin, dtype=np.uint16, count=-1)
            for i in range(0, len(data), n):
                yield data[i], data[i+1:i+n]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        iword, owords = self.data[idx]
        return int(iword), np.array(owords, dtype=np.int)


def train(args):
    d = Dictionary.load(args.vocab)
    wf = np.array(d.count)
    wf[wf == 0] = 1
    wf = wf / wf.sum()
    ws = 1 - np.sqrt(args.ss_t / wf)
    ws = np.clip(ws, 0, 1)
    vocab_size = len(d)
    weights = wf if args.weights else None
    if not os.path.isdir(args.save_dir):
        os.makedirs(args.save_dir)
    model = Word2Vec(vocab_size=vocab_size, embedding_size=args.e_dim)
    modelpath = os.path.join(args.save_dir, '{}.pt'.format(args.name))
    sgns = SGNS(embedding=model, vocab_size=vocab_size, n_negs=args.n_negs, weights=weights, pad=d.unk())
    if os.path.isfile(modelpath) and args.conti:
        sgns.load_state_dict(t.load(modelpath))
    if args.cuda:
        sgns = sgns.cuda()
    optim = Adam(sgns.parameters())
    optimpath = os.path.join(args.save_dir, '{}.optim.pt'.format(args.name))
    if os.path.isfile(optimpath) and args.conti:
        optim.load_state_dict(t.load(optimpath))
    dataset = PermutedSubsampledCorpus(args.data, ws=ws)
    dataloader = DataLoader(dataset, batch_size=args.mb, shuffle=True, num_workers=0)
    for epoch in range(1, args.epoch + 1):
        total_batches = int(np.ceil(len(dataset) / args.mb))
        pbar = tqdm(dataloader)
        pbar.set_description("[Epoch {}]".format(epoch))
        for iword, owords in pbar:
            loss = sgns(iword, owords)
            optim.zero_grad()
            loss.backward()
            optim.step()
            pbar.set_postfix(loss=loss.item())

        t.save(sgns.state_dict(), os.path.join(args.save_dir, '{}-e{}.pt'.format(args.name, epoch)))
        t.save(optim.state_dict(), os.path.join(args.save_dir, '{}-e{}.optim.pt'.format(args.name, epoch)))


if __name__ == '__main__':
    train(parse_args())





# Retriever


In [0]:
%%writefile retriever.py
import argparse
import os, sys
import numpy as np
import time
import pickle
from functools import total_ordering
from sklearn.feature_extraction.text import TfidfVectorizer

from .utils import sentence_iterator, Word

import logging
logger = logging.getLogger('pungen')

# define rich comparison ordering methods
@total_ordering
class Template(object):
    def __init__(self, tokens, keyword, id_):
        self.id = int(id_)
        self.tokens = tokens
        self.keyword_positions = [i for i, w in enumerate(tokens) if w == keyword]
        self.num_key = len(self.keyword_positions)
        # the key_word id is the largest index in keyword_positions, or the last instance of the word
        self.keyword_id = None if self.num_key == 0 else max(self.keyword_positions)

    def __len__(self):
        return len(self.tokens)

    # the swap part of retrieve + swap
    def replace_keyword(self, word):
        tokens = list(self.tokens)
        tokens[self.keyword_id] = word
        return tokens

    def __str__(self):
        return ' '.join(['[{}]'.format(w) if i == self.keyword_id else w for i, w in enumerate(self.tokens)])

    # for ranking the words by the placement in the sentence
    def __lt__(self, other):
        # Containing keyword is better
        if self.num_key == 0:
            return True
        # Fewer keywords is better
        if self.num_key > other.num_key:
            return True
        # Later keywords is better
        if self.keyword_id < other.keyword_id:
            return True
        return False

    def __eq__(self, other):
        if self.num_key == 0 and other.num_key == 0:
            return True
        if self.num_key == other.num_key and self.keyword_id == other.keyword_id:
            return True
        return False


class Retriever(object):
    def __init__(self, doc_files, path=None, overwrite=False):
        logger.info('reading retriever docs from {}'.format(' '.join(doc_files)))
        self.docs = [line.strip() for line in open(doc_files[0], 'r')]

        if overwrite or (path is None or not os.path.exists(path)):
            logger.info('building retriever index')
            self.vectorizer = TfidfVectorizer(analyzer=str.split)
            self.tfidf_matrix = self.vectorizer.fit_transform(self.docs)
            if path is not None:
                self.save(path)
        else:
            logger.info('loading retriever index from {}'.format(path))
            with open(path, 'rb') as fin:
                obj = pickle.load(fin)
                self.vectorizer = obj['vectorizer']
                self.tfidf_matrix = obj['tfidf_mat']

    def save(self, path):
        with open(path, 'wb') as fout:
            obj = {
                    'vectorizer': self.vectorizer,
                    'tfidf_mat': self.tfidf_matrix,
                    }
            pickle.dump(obj, fout)

    def query(self, keywords, k=1):
        features = self.vectorizer.transform([keywords])
        scores = self.tfidf_matrix * features.T
        scores = scores.todense()
        scores = np.squeeze(np.array(scores), axis=1)
        ids = np.argsort(scores)[-k:][::-1]
        return ids

    def valid_template(self, template):
        return template.num_key == 1

    def retrieve_pun_template(self, alter_word, len_threshold=10, pos_threshold=0.5, num_cands=500, num_templates=None):
        ids = self.query(alter_word, num_cands)
        templates = [Template(self.docs[id_].split(), alter_word, id_) for id_ in ids]
        templates = [t for t in templates if t.num_key > 0 and len(t.tokens) > len_threshold]
        if len(templates) == 0:
            logger.info('FAIL: no retrieved sentence contains the keyword {}.'.format(alter_word))
            return []

        valid_templates = [t for t in templates if self.valid_template(t)]
        if len(valid_templates) == 0:
            valid_templates = templates
        templates = sorted(valid_templates, reverse=True)[:num_templates]
        return templates


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--doc-file', nargs='+', help='training corpus')
    parser.add_argument('--interactive', action='store_true')
    parser.add_argument('--path', default='models/retriever.pkl', help='retriever model path')
    parser.add_argument('--overwrite', action='store_true')
    parser.add_argument('--keywords', help='file containing keywords')
    parser.add_argument('--alterwords', help='file containing alternative words')
    parser.add_argument('--outfile', help='output file for the retrieved sentences')
    args = parser.parse_args()

    retriever = Retriever(args.doc_file, args.path, args.overwrite)

    if args.interactive:
        while True:
            alter_word, pun_word = input('Keywords:\n').split()
            alter_sents, pun_sents, pun_word_ids, alter_ori_sents = retriever.retrieve_pun_template(alter_word, num_cands=100)
            for ori_sent, sent in zip(alter_ori_sents, alter_sents):
                print(' '.join(sent))
                print(' '.join(ori_sent))
            if not alter_sents:
                print('No candidates found')
    elif args.outfile:
        with open(args.keywords, 'r') as fin, open(args.alterwords, 'r') as afin, open(args.outfile, 'w') as outf:
            for key, alter in zip(fin, afin):
                key = key.strip()
                alter = alter.strip()
                ids = retriever.query(key, 10)
                contents = [retriever.ori_docs[id_] for id_ in ids]
                contents.sort(key = lambda s: len(s), reverse=True)
#                print(line.strip())
                for ct in contents:
                    ct_list = ct.split()
                    try:
                        idx = ct_list.index(key)
                    except:
                        sys.stderr.write('cannot find the word %s!\n' % key)
                        continue
                    ct_list[idx] = alter
                    outf.write(ct.lower() + '\n')
                    print(' '.join(ct_list).lower())
                    break


# Preprocess for edit

In [0]:
%%writefile preprocess_pungen.py
#!/usr/bin/env python3
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the LICENSE file in
# the root directory of this source tree. An additional grant of patent rights
# can be found in the PATENTS file in the same directory.
"""
Data pre-processing: build vocabularies and binarize training data.
"""

import argparse
from collections import Counter
from itertools import zip_longest
import os
import shutil

from fairseq.data import indexed_dataset, EditDictionary as Dictionary
from fairseq.tokenizer import Tokenizer, tokenize_line
from multiprocessing import Pool, Manager, Process


def get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', '--source-lang', default=None, metavar='SRC', help='source language')
    parser.add_argument('-t', '--target-lang', default=None, metavar='TARGET', help='target language')
    parser.add_argument('--trainpref', metavar='FP', default=None, help='train file prefix')
    parser.add_argument('--validpref', metavar='FP', default=None, help='comma separated, valid file prefixes')
    parser.add_argument('--testpref', metavar='FP', default=None, help='comma separated, test file prefixes')
    parser.add_argument('--destdir', metavar='DIR', default='data-bin', help='destination dir')
    parser.add_argument('--thresholdtgt', metavar='N', default=0, type=int,
                        help='map words appearing less than threshold times to unknown')
    parser.add_argument('--thresholdsrc', metavar='N', default=0, type=int,
                        help='map words appearing less than threshold times to unknown')
    parser.add_argument('--tgtdict', metavar='FP', help='reuse given target dictionary')
    parser.add_argument('--srcdict', metavar='FP', help='reuse given source dictionary')
    parser.add_argument('--nwordstgt', metavar='N', default=-1, type=int, help='number of target words to retain')
    parser.add_argument('--nwordssrc', metavar='N', default=-1, type=int, help='number of source words to retain')
    parser.add_argument('--alignfile', metavar='ALIGN', default=None, help='an alignment file (optional)')
    parser.add_argument('--output-format', metavar='FORMAT', default='binary', choices=['binary', 'raw'],
                        help='output format (optional)')
    parser.add_argument('--joined-dictionary', action='store_true', help='Generate joined dictionary')
    parser.add_argument('--only-source', action='store_true', help='Only process the source language')
    parser.add_argument('--padding-factor', metavar='N', default=8, type=int,
                        help='Pad dictionary size to be multiple of N')
    parser.add_argument('--workers', metavar='N', default=1, type=int, help='number of parallel workers')
    return parser


def main(args):
    print(args)
    os.makedirs(args.destdir, exist_ok=True)
    target = not args.only_source

    def build_dictionary(filenames):
        d = Dictionary()
        for filename in filenames:
            Tokenizer.add_file_to_dictionary(filename, d, tokenize_line, args.workers)
        return d

    def train_path(lang):
        return '{}{}'.format(args.trainpref, ('.' + lang) if lang else '')

    def file_name(prefix, lang):
        fname = prefix
        if lang is not None:
            fname += f'.{lang}'
        return fname

    def dest_path(prefix, lang):
        return os.path.join(args.destdir, file_name(prefix, lang))

    def dict_path(lang):
        return dest_path('dict', lang) + '.txt'

    if args.joined_dictionary:
        assert not args.srcdict, 'cannot combine --srcdict and --joined-dictionary'
        assert not args.tgtdict, 'cannot combine --tgtdict and --joined-dictionary'
        src_dict = build_dictionary(set([
            train_path(lang)
            for lang in [args.source_lang, args.target_lang]
        ]))
        tgt_dict = src_dict
    else:
        if args.srcdict:
            src_dict = Dictionary.load(args.srcdict)
        else:
            assert args.trainpref, "--trainpref must be set if --srcdict is not specified"
            src_dict = build_dictionary([train_path(args.source_lang)])
        if target:
            if args.tgtdict:
                tgt_dict = Dictionary.load(args.tgtdict)
            else:
                assert args.trainpref, "--trainpref must be set if --tgtdict is not specified"
                tgt_dict = build_dictionary([train_path(args.target_lang)])

    src_dict.finalize(
        threshold=args.thresholdsrc,
        nwords=args.nwordssrc,
        padding_factor=args.padding_factor,
    )
    src_dict.save(dict_path(args.source_lang))
    if target:
        if not args.joined_dictionary:
            tgt_dict.finalize(
                threshold=args.thresholdtgt,
                nwords=args.nwordstgt,
                padding_factor=args.padding_factor,
            )
        tgt_dict.save(dict_path(args.target_lang))

    def make_binary_dataset(input_prefix, output_prefix, lang, num_workers):
        dict = Dictionary.load(dict_path(lang))
        print('| [{}] Dictionary: {} types'.format(lang, len(dict) - 1))
        n_seq_tok = [0, 0]
        replaced = Counter()

        def merge_result(worker_result):
            replaced.update(worker_result['replaced'])
            n_seq_tok[0] += worker_result['nseq']
            n_seq_tok[1] += worker_result['ntok']

        input_file = '{}{}'.format(input_prefix, ('.' + lang) if lang is not None else '')
        offsets = Tokenizer.find_offsets(input_file, num_workers)
        pool = None
        if num_workers > 1:
            pool = Pool(processes=num_workers-1)
            for worker_id in range(1, num_workers):
                prefix = "{}{}".format(output_prefix, worker_id)
                pool.apply_async(binarize, (args, input_file, dict, prefix, lang,
                                            offsets[worker_id],
                                            offsets[worker_id + 1]), callback=merge_result)
            pool.close()

        ds = indexed_dataset.IndexedDatasetBuilder(dataset_dest_file(args, output_prefix, lang, 'bin'))
        merge_result(Tokenizer.binarize(input_file, dict, lambda t: ds.add_item(t),
                                        offset=0, end=offsets[1]))
        if num_workers > 1:
            pool.join()
            for worker_id in range(1, num_workers):
                prefix = "{}{}".format(output_prefix, worker_id)
                temp_file_path = dataset_dest_prefix(args, prefix, lang)
                ds.merge_file_(temp_file_path)
                os.remove(indexed_dataset.data_file_path(temp_file_path))
                os.remove(indexed_dataset.index_file_path(temp_file_path))


        ds.finalize(dataset_dest_file(args, output_prefix, lang, 'idx'))


        print('| [{}] {}: {} sents, {} tokens, {:.3}% replaced by {}'.format(
            lang, input_file, n_seq_tok[0], n_seq_tok[1],
            100 * sum(replaced.values()) / n_seq_tok[1], dict.unk_word))



    def make_dataset(input_prefix, output_prefix, lang, num_workers=1):
        if args.output_format == 'binary':
            make_binary_dataset(input_prefix, output_prefix, lang, num_workers)
        elif args.output_format == 'raw':
            # Copy original text file to destination folder
            output_text_file = dest_path(
                output_prefix + '.{}-{}'.format(args.source_lang, args.target_lang),
                lang,
            )
            shutil.copyfile(file_name(input_prefix, lang), output_text_file)

    def make_all(lang):
        if args.trainpref:
            make_dataset(args.trainpref, 'train', lang, num_workers=args.workers)
        if args.validpref:
            for k, validpref in enumerate(args.validpref.split(',')):
                outprefix = 'valid{}'.format(k) if k > 0 else 'valid'
                make_dataset(validpref, outprefix, lang)
        if args.testpref:
            for k, testpref in enumerate(args.testpref.split(',')):
                outprefix = 'test{}'.format(k) if k > 0 else 'test'
                make_dataset(testpref, outprefix, lang)

    make_all(args.source_lang)
    if target:
        make_all(args.target_lang)

    print('| Wrote preprocessed data to {}'.format(args.destdir))

    if args.alignfile:
        assert args.trainpref, "--trainpref must be set if --alignfile is specified"
        src_file_name = train_path(args.source_lang)
        tgt_file_name = train_path(args.target_lang)
        src_dict = Dictionary.load(dict_path(args.source_lang))
        tgt_dict = Dictionary.load(dict_path(args.target_lang))
        freq_map = {}
        with open(args.alignfile, 'r') as align_file:
            with open(src_file_name, 'r') as src_file:
                with open(tgt_file_name, 'r') as tgt_file:
                    for a, s, t in zip_longest(align_file, src_file, tgt_file):
                        si = Tokenizer.tokenize(s, src_dict, add_if_not_exist=False)
                        ti = Tokenizer.tokenize(t, tgt_dict, add_if_not_exist=False)
                        ai = list(map(lambda x: tuple(x.split('-')), a.split()))
                        for sai, tai in ai:
                            srcidx = si[int(sai)]
                            tgtidx = ti[int(tai)]
                            if srcidx != src_dict.unk() and tgtidx != tgt_dict.unk():
                                assert srcidx != src_dict.pad()
                                assert srcidx != src_dict.eos()
                                assert tgtidx != tgt_dict.pad()
                                assert tgtidx != tgt_dict.eos()

                                if srcidx not in freq_map:
                                    freq_map[srcidx] = {}
                                if tgtidx not in freq_map[srcidx]:
                                    freq_map[srcidx][tgtidx] = 1
                                else:
                                    freq_map[srcidx][tgtidx] += 1

        align_dict = {}
        for srcidx in freq_map.keys():
            align_dict[srcidx] = max(freq_map[srcidx], key=freq_map[srcidx].get)

        with open(os.path.join(args.destdir, 'alignment.{}-{}.txt'.format(
                args.source_lang, args.target_lang)), 'w') as f:
            for k, v in align_dict.items():
                print('{} {}'.format(src_dict[k], tgt_dict[v]), file=f)



def binarize(args, filename, dict, output_prefix, lang, offset, end):

    ds = indexed_dataset.IndexedDatasetBuilder(dataset_dest_file(args, output_prefix, lang, 'bin'))
    def consumer(tensor):
        ds.add_item(tensor)

    res = Tokenizer.binarize(filename, dict, consumer, offset=offset, end=end)
    ds.finalize(dataset_dest_file(args, output_prefix, lang, 'idx'))
    return res

def dataset_dest_prefix(args, output_prefix, lang):
    base = f'{args.destdir}/{output_prefix}'
    lang_part = f'.{args.source_lang}-{args.target_lang}.{lang}' if lang is not None else ''
    return f'{base}{lang_part}'


def dataset_dest_file(args, output_prefix, lang, extension):
    base = dataset_dest_prefix(args, output_prefix, lang)
    return f'{base}.{extension}'


if __name__ == '__main__':
    parser = get_parser()
    args = parser.parse_args()
    main(args)


# Training for edit

In [0]:
#!/usr/bin/env python3 -u
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the LICENSE file in
# the root directory of this source tree. An additional grant of patent rights
# can be found in the PATENTS file in the same directory.
"""
Train a new model on one or across multiple GPUs.
"""

import collections
import itertools
import os
import math
import torch

from fairseq import distributed_utils, options, progress_bar, tasks, utils
from fairseq.data import iterators
from fairseq.trainer import Trainer
from fairseq.meters import AverageMeter, StopwatchMeter


def main(args):
    if args.max_tokens is None:
        args.max_tokens = 6000
    print(args)

    if not torch.cuda.is_available():
        raise NotImplementedError('Training on CPU is not supported')
    torch.cuda.set_device(args.device_id)
    torch.manual_seed(args.seed)

    # Setup task, e.g., translation, language modeling, etc.
    task = tasks.setup_task(args)

    # Load dataset splits
    load_dataset_splits(task, ['train', 'valid'])

    # Build model and criterion
    model = task.build_model(args)
    criterion = task.build_criterion(args)
    print('| model {}, criterion {}'.format(args.arch, criterion.__class__.__name__))
    print('| num. model params: {}'.format(sum(p.numel() for p in model.parameters())))

    # Make a dummy batch to (i) warm the caching allocator and (ii) as a
    # placeholder DistributedDataParallel when there's an uneven number of
    # batches per worker.
    max_positions = utils.resolve_max_positions(
        task.max_positions(),
        model.max_positions(),
    )
    dummy_batch = task.dataset('train').get_dummy_batch(args.max_tokens, max_positions)

    # Build trainer
    trainer = Trainer(args, task, model, criterion, dummy_batch)
    print('| training on {} GPUs'.format(args.distributed_world_size))
    print('| max tokens per GPU = {} and max sentences per GPU = {}'.format(
        args.max_tokens,
        args.max_sentences,
    ))

    # Initialize dataloader
    epoch_itr = task.get_batch_iterator(
        dataset=task.dataset(args.train_subset),
        max_tokens=args.max_tokens,
        max_sentences=args.max_sentences,
        max_positions=max_positions,
        ignore_invalid_inputs=True,
        required_batch_size_multiple=8,
        seed=args.seed,
        num_shards=args.distributed_world_size,
        shard_id=args.distributed_rank,
    )

    # Load the latest checkpoint if one is available
    if not load_checkpoint(args, trainer, epoch_itr):
        trainer.dummy_train_step([dummy_batch])

    # Train until the learning rate gets too small
    max_epoch = args.max_epoch or math.inf
    max_update = args.max_update or math.inf
    lr = trainer.get_lr()
    train_meter = StopwatchMeter()
    train_meter.start()
    valid_losses = [None]
    valid_subsets = args.valid_subset.split(',')
    while lr > args.min_lr and epoch_itr.epoch < max_epoch and trainer.get_num_updates() < max_update:
        # train for one epoch
        train(args, trainer, task, epoch_itr)

        if epoch_itr.epoch % args.validate_interval == 0:
            valid_losses = validate(args, trainer, task, epoch_itr, valid_subsets)

        # only use first validation loss to update the learning rate
        lr = trainer.lr_step(epoch_itr.epoch, valid_losses[0])

        # save checkpoint
        if epoch_itr.epoch % args.save_interval == 0:
            save_checkpoint(args, trainer, epoch_itr, valid_losses[0])
    train_meter.stop()
    print('| done training in {:.1f} seconds'.format(train_meter.sum))


def train(args, trainer, task, epoch_itr):
    """Train the model for one epoch."""

    # Update parameters every N batches
    if epoch_itr.epoch <= len(args.update_freq):
        update_freq = args.update_freq[epoch_itr.epoch - 1]
    else:
        update_freq = args.update_freq[-1]

    # Initialize data iterator
    itr = epoch_itr.next_epoch_itr(fix_batches_to_gpus=args.fix_batches_to_gpus)
    itr = iterators.GroupedIterator(itr, update_freq)
    progress = progress_bar.build_progress_bar(
        args, itr, epoch_itr.epoch, no_progress_bar='simple',
    )

    extra_meters = collections.defaultdict(lambda: AverageMeter())
    first_valid = args.valid_subset.split(',')[0]
    max_update = args.max_update or math.inf
    for i, samples in enumerate(progress, start=epoch_itr.iterations_in_epoch):
        log_output = trainer.train_step(samples)
        if log_output is None:
            continue

        # log mid-epoch stats
        stats = get_training_stats(trainer)
        for k, v in log_output.items():
            if k in ['loss', 'nll_loss', 'ntokens', 'nsentences', 'sample_size']:
                continue  # these are already logged above
            if 'loss' in k:
                extra_meters[k].update(v, log_output['sample_size'])
            else:
                extra_meters[k].update(v)
            stats[k] = extra_meters[k].avg
        progress.log(stats)

        # ignore the first mini-batch in words-per-second calculation
        if i == 0:
            trainer.get_meter('wps').reset()

        num_updates = trainer.get_num_updates()
        if args.save_interval_updates > 0 and num_updates % args.save_interval_updates == 0 and num_updates > 0:
            valid_losses = validate(args, trainer, task, epoch_itr, [first_valid])
            save_checkpoint(args, trainer, epoch_itr, valid_losses[0])

        if num_updates >= max_update:
            break

    # log end-of-epoch stats
    stats = get_training_stats(trainer)
    for k, meter in extra_meters.items():
        stats[k] = meter.avg
    progress.print(stats)

    # reset training meters
    for k in [
        'train_loss', 'train_nll_loss', 'wps', 'ups', 'wpb', 'bsz', 'gnorm', 'clip',
    ]:
        meter = trainer.get_meter(k)
        if meter is not None:
            meter.reset()


def get_training_stats(trainer):
    stats = collections.OrderedDict()
    stats['loss'] = '{:.3f}'.format(trainer.get_meter('train_loss').avg)
    if trainer.get_meter('train_nll_loss').count > 0:
        nll_loss = trainer.get_meter('train_nll_loss').avg
        stats['nll_loss'] = '{:.3f}'.format(nll_loss)
    else:
        nll_loss = trainer.get_meter('train_loss').avg
    stats['ppl'] = get_perplexity(nll_loss)
    stats['wps'] = round(trainer.get_meter('wps').avg)
    stats['ups'] = '{:.1f}'.format(trainer.get_meter('ups').avg)
    stats['wpb'] = round(trainer.get_meter('wpb').avg)
    stats['bsz'] = round(trainer.get_meter('bsz').avg)
    stats['num_updates'] = trainer.get_num_updates()
    stats['lr'] = trainer.get_lr()
    stats['gnorm'] = '{:.3f}'.format(trainer.get_meter('gnorm').avg)
    stats['clip'] = '{:.0%}'.format(trainer.get_meter('clip').avg)
    stats['oom'] = trainer.get_meter('oom').avg
    if trainer.get_meter('loss_scale') is not None:
        stats['loss_scale'] = '{:.3f}'.format(trainer.get_meter('loss_scale').avg)
    stats['wall'] = round(trainer.get_meter('wall').elapsed_time)
    stats['train_wall'] = round(trainer.get_meter('train_wall').sum)
    return stats


def validate(args, trainer, task, epoch_itr, subsets):
    """Evaluate the model on the validation set(s) and return the losses."""
    valid_losses = []
    for subset in subsets:
        # Initialize data iterator
        itr = task.get_batch_iterator(
            dataset=task.dataset(subset),
            max_tokens=args.max_tokens,
            max_sentences=args.max_sentences_valid,
            max_positions=utils.resolve_max_positions(
                task.max_positions(),
                trainer.get_model().max_positions(),
            ),
            ignore_invalid_inputs=args.skip_invalid_size_inputs_valid_test,
            required_batch_size_multiple=8,
            seed=args.seed,
            num_shards=args.distributed_world_size,
            shard_id=args.distributed_rank,
        ).next_epoch_itr(shuffle=False)
        progress = progress_bar.build_progress_bar(
            args, itr, epoch_itr.epoch,
            prefix='valid on \'{}\' subset'.format(subset),
            no_progress_bar='simple'
        )

        # reset validation loss meters
        for k in ['valid_loss', 'valid_nll_loss']:
            meter = trainer.get_meter(k)
            if meter is not None:
                meter.reset()
        extra_meters = collections.defaultdict(lambda: AverageMeter())

        for sample in progress:
            log_output = trainer.valid_step(sample)

            for k, v in log_output.items():
                if k in ['loss', 'nll_loss', 'ntokens', 'nsentences', 'sample_size']:
                    continue
                extra_meters[k].update(v)

        # log validation stats
        stats = get_valid_stats(trainer)
        for k, meter in extra_meters.items():
            stats[k] = meter.avg
        progress.print(stats)

        valid_losses.append(stats['valid_loss'])
    return valid_losses


def get_valid_stats(trainer):
    stats = collections.OrderedDict()
    stats['valid_loss'] = trainer.get_meter('valid_loss').avg
    if trainer.get_meter('valid_nll_loss').count > 0:
        nll_loss = trainer.get_meter('valid_nll_loss').avg
        stats['valid_nll_loss'] = nll_loss
    else:
        nll_loss = trainer.get_meter('valid_loss').avg
    stats['valid_ppl'] = get_perplexity(nll_loss)
    stats['num_updates'] = trainer.get_num_updates()
    if hasattr(save_checkpoint, 'best'):
        stats['best'] = min(save_checkpoint.best, stats['valid_loss'])
    return stats


def get_perplexity(loss):
    try:
        return '{:.2f}'.format(math.pow(2, loss))
    except OverflowError:
        return float('inf')


def save_checkpoint(args, trainer, epoch_itr, val_loss):
    if args.no_save or not distributed_utils.is_master(args):
        return
    epoch = epoch_itr.epoch
    end_of_epoch = epoch_itr.end_of_epoch()
    updates = trainer.get_num_updates()

    checkpoint_conds = collections.OrderedDict()
    checkpoint_conds['checkpoint{}.pt'.format(epoch)] = (
            end_of_epoch and not args.no_epoch_checkpoints and
            epoch % args.save_interval == 0
    )
    checkpoint_conds['checkpoint_{}_{}.pt'.format(epoch, updates)] = (
            not end_of_epoch and args.save_interval_updates > 0 and
            updates % args.save_interval_updates == 0
    )
    checkpoint_conds['checkpoint_best.pt'] = (
            val_loss is not None and
            (not hasattr(save_checkpoint, 'best') or val_loss < save_checkpoint.best)
    )
    checkpoint_conds['checkpoint_last.pt'] = True  # keep this last so that it's a symlink

    prev_best = getattr(save_checkpoint, 'best', val_loss)
    if val_loss is not None:
        save_checkpoint.best = min(val_loss, prev_best)
    extra_state = {
        'best': save_checkpoint.best,
        'train_iterator': epoch_itr.state_dict(),
        'val_loss': val_loss,
    }

    checkpoints = [os.path.join(args.save_dir, fn) for fn, cond in checkpoint_conds.items() if cond]
    if len(checkpoints) > 0:
        for cp in checkpoints:
            trainer.save_checkpoint(cp, extra_state)

    if not end_of_epoch and args.keep_interval_updates > 0:
        # remove old checkpoints; checkpoints are sorted in descending order
        checkpoints = utils.checkpoint_paths(args.save_dir, pattern=r'checkpoint_\d+_(\d+)\.pt')
        for old_chk in checkpoints[args.keep_interval_updates:]:
            os.remove(old_chk)


def load_checkpoint(args, trainer, epoch_itr):
    """Load a checkpoint and replay dataloader to match."""
    os.makedirs(args.save_dir, exist_ok=True)
    checkpoint_path = os.path.join(args.save_dir, args.restore_file)
    if os.path.isfile(checkpoint_path):
        extra_state = trainer.load_checkpoint(checkpoint_path, args.reset_optimizer, args.reset_lr_scheduler,
                                              eval(args.optimizer_overrides))
        if extra_state is not None:
            # replay train iterator to match checkpoint
            epoch_itr.load_state_dict(extra_state['train_iterator'])

            print('| loaded checkpoint {} (epoch {} @ {} updates)'.format(
                checkpoint_path, epoch_itr.epoch, trainer.get_num_updates()))

            trainer.lr_step(epoch_itr.epoch)
            trainer.lr_step_update(trainer.get_num_updates())
            if 'best' in extra_state:
                save_checkpoint.best = extra_state['best']
        return True
    return False


def load_dataset_splits(task, splits):
    for split in splits:
        if split == 'train':
            task.load_dataset(split, combine=True)
        else:
            for k in itertools.count():
                split_k = split + (str(k) if k > 0 else '')
                try:
                    task.load_dataset(split_k, combine=False)
                except FileNotFoundError as e:
                    if k > 0:
                        break
                    raise e


if __name__ == '__main__':
    parser = options.get_training_parser()
    args = options.parse_args_and_arch(parser)

    if args.distributed_port > 0 or args.distributed_init_method is not None:
        from distributed_train import main as distributed_main

        distributed_main(args)
    elif args.distributed_world_size > 1:
        from multiprocessing_train import main as multiprocessing_main

        multiprocessing_main(args)
    else:
        main(args)


# Eval funniness

In [0]:
%%writefile eval_scoring_func.py
"""Compute the correlation of funniness scorers given by a scoring function and those
given by humans.
"""

import os
import argparse
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import spearmanr, zscore
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

from fairseq.data.dictionary import Dictionary

from pungen.scorer import LMScorer, SurprisalScorer, UnigramModel, GoodmanScorer
from pungen.options import add_scorer_args, add_generic_args
from pungen.utils import logging_config, get_spacy_nlp
from pungen.wordvec.generate import SkipGram
from pungen.pretrained_wordvec import Glove

import logging
logger = logging.getLogger('pungen')

nlp = get_spacy_nlp(tokenizer='default', disable=['tagger', 'ner', 'parser'])

def parse_args():
    parser = argparse.ArgumentParser()
    add_scorer_args(parser)
    add_generic_args(parser)
    parser.add_argument('--skipgram-model', nargs=2, help='pretrained skipgram model [vocab, model]')
    parser.add_argument('--skipgram-embed-size', type=int, default=300, help='word embedding size in skipgram model')
    parser.add_argument('--human-eval', help='path to human score file')
    parser.add_argument('--features', nargs='+', default=['ratio', 'grammar', 'ambiguity'], help='features to analyze')
    parser.add_argument('--ignore-cache', action='store_true', help='ignore cached scores. cache path: `args.outdir`/scores.json')
    parser.add_argument('--analysis', action='store_true', help='using analysis data instead of generated data')
    parser.add_argument('--tokenized', action='store_true', help='whether the sentences are tokenized')
    args = parser.parse_args()
    return args


def linear_regression(scores, features, target='human'):
    X = np.array([[s[f] for f in features] for s in scores])
    y = np.array([s[target] for s in scores])
    model = LinearRegression().fit(X, y)
    coeffs = {k: v for k, v in zip(features, model.coef_)}
    r2 = model.score(X, y)
    f_scores, p_values = f_regression(X, y)
    f_scores = {k: {'coeff': c, 'fscore': f, 'pvalue': p}
            for k, c, f, p in zip(features, model.coef_, f_scores, p_values)}
    return model, r2, f_scores

def plot(scores, x_features, y_features, path='./', file_format='jpg'):
    plt.figure(1)
    ind = 0
    nrows = len(x_features)
    ncols = len(y_features)
    f, axarr = plt.subplots(nrows, ncols)
    for i, x_feature in enumerate(x_features):
        for j, y_feature in enumerate(y_features):
            x = np.array([s[x_feature] for s in scores])
            y = np.array([s[y_feature] for s in scores])
            axarr[ind].scatter(x, y)
            axarr[ind].set_xlabel(x_feature)
            axarr[ind].set_ylabel(y_feature)
            ind += 1
    f.subplots_adjust(hspace=0.8)
    plt.savefig('x_human.jpg', format=file_format)

def parse_human_eval_data(path, tokenized):
    candidates = []
    with open(path) as fin:
        for line in fin:
            ss = line.strip().split('\t')
            if tokenized:
                sent = ss[0].split()
            else:
                sent = [x.text for x in nlp(ss[0])]
            sent = [x.lower() for x in sent]
            pun_word, alter_word = ss[1].split('-')
            method = ss[2]
            score = float(ss[3])
            # TODO: move to preprocess
            try:
                pun_word_id = sent.index(pun_word)
            except ValueError:
                continue
            c = {
                    'pun_sent': sent,
                    'pun_word_id': pun_word_id,
                    'alter_word': alter_word,
                    'scores': {'human': score},
                    'type': method,
                }
            candidates.append(c)
    return candidates

def score_examples(args):
    lm = LMScorer.load_model(args.lm_path)
    unigram_model = UnigramModel(args.word_counts_path, args.oov_prob)
    skipgram = SkipGram.load_model(args.skipgram_model[0], args.skipgram_model[1], embedding_size=args.skipgram_embed_size, cpu=args.cpu)

    candidates = parse_human_eval_data(args.human_eval, args.tokenized)

    #vocab = build_vocab(candidates)
    #glove = Glove.from_file('/u/scr/nlp/data/glove_vecs/glove.840B.300d.txt', vocab)
    glove = None

    scorers = [SurprisalScorer(lm, unigram_model, local_window_size=args.local_window_size),
               GoodmanScorer(unigram_model, skipgram, glove)]
    #scorers = [GoodmanScorer(unigram_model, skipgram, glove)]

    for c in candidates:
        for scorer in scorers:
            scores = scorer.analyze(c['pun_sent'], c['pun_word_id'], c['alter_word'])
            c['scores'].update(scores)
    return candidates

def build_vocab(candidates):
    d = Dictionary()
    for c in candidates:
        for w in c['pun_sent']:
            d.add_symbol(w)
    return d

def compute_stats(candidates):
    results_by_type = defaultdict(lambda : defaultdict(float))
    for c in candidates:
        results_by_type[c['type']]['count'] += 1
        results_by_type[c['type']]['score'] += c['scores']['human']
    logger.info('Stats')
    str_format = '{type:<10s}{count:10d}{score:10.2f}'
    for k, v in results_by_type.items():
        logger.info(str_format.format(type=k, count=int(v['count']), score=v['score'] / v['count']))


def main(args):
    json.dump(vars(args), open(os.path.join(args.outdir, 'config.json'), 'w'))

    filename = os.path.join(args.outdir, 'scores.json')
    if not os.path.exists(filename) or args.ignore_cache:
        candidates = score_examples(args)
        json.dump(candidates, open(filename, 'w'))
    else:
        candidates = json.load(open(filename))

    candidates = [c for c in candidates if c['type'] in ('pun', 'depun', 'retrieved_pw', 'retrieved_aw', 'nonpun')]

    compute_stats(candidates)

    # Normalization, cutoff 2-\sigma
    features = candidates[0]['scores'].keys()
    for feat in features:
        if feat != 'human':
            _scores = zscore([c['scores'][feat] for c in candidates])
            for _score, c in zip(_scores, candidates):
                if _score > 0:
                    _score = min(2, _score)
                elif _score < 0:
                    _score = max(-2, _score)
                c['scores'][feat] = _score

    # Correlation
    if args.analysis:
        all_types = tuple(set([c['type'] for c in candidates]))
        for types in [('pun',), ('pun', 'depun'), ('pun', 'nonpun', 'retrieved_pw', 'retrieved_aw')]:
            _candidates = [c for c in candidates if c['type'] in types]
            human_scores = [c['scores']['human'] for c in _candidates]
            logger.info('correlation for {} sentences of types {}'.format(len(_candidates), str(types)))
            for metric in _candidates[0]['scores'].keys():
                if metric == 'human':
                    continue
                _scores = [c['scores'][metric] for c in _candidates]
                corr = spearmanr(human_scores, _scores)
                logger.info('{:<15s}: {:>8.4f} p={:>8.4f}'.format(metric, corr.correlation, corr.pvalue))

    # All scores
    scores = [c['scores'] for c in candidates]

    # Plot
    #plot(scores, x_features=args.features, y_features=['human'])

    # Linear regression
    features = args.features
    scores = [c['scores'] for c in candidates]
    model, r2, feature_stats = linear_regression(scores, features)

    logger.info('linear regression')
    for name, stats in feature_stats.items():
        stats_format = '{name:<15s}' + ' '.join(['{}={{{}:>8.4f}}'.format(s, s) for s in stats.keys()])
        logger.info(stats_format.format(name=name, **stats))
    logger.info('R^2={:.4f}'.format(r2))

if __name__ == '__main__':
    args = parse_args()
    logging_config(os.path.join(args.outdir, 'console.log'), console_level=logging.DEBUG)
    main(args)


# Generate puns

In [0]:
%%writefile generate_pun.py 
import os
import pickle
import argparse
import json
from collections import defaultdict
import fuzzy

from fairseq import options

from pungen.retriever import Retriever
from pungen.generator import SkipGram, RulebasedGenerator, NeuralCombinerGenerator, RetrieveGenerator, RetrieveSwapGenerator, KeywordsGenerator
from pungen.scorer import LMScorer, SurprisalScorer, UnigramModel, RandomScorer, GoodmanScorer
from pungen.type import TypeRecognizer
from pungen.options import add_scorer_args, add_editor_args, add_retriever_args, add_generic_args, add_type_checker_args
from pungen.utils import logging_config, get_lemma, ensure_exist, get_spacy_nlp

import logging
logger = logging.getLogger('pungen')

nlp = get_spacy_nlp()

def parse_args():
    parser = options.get_generation_parser(interactive=True)
    add_scorer_args(parser)
    add_editor_args(parser)
    add_retriever_args(parser)
    add_type_checker_args(parser)
    add_generic_args(parser)
    parser.add_argument('--pun-words')
    parser.add_argument('--system', default='rule')
    parser.add_argument('--max-num-examples', type=int, default=-1)
    args = options.parse_args_and_arch(parser)
    return args

def iter_keywords(file_):
    with open(file_, 'r') as fin:
        for line in fin:
            alter_word, pun_word = line.strip().split()
            yield alter_word, pun_word

def feasible_pun_words(pun_word, alter_word, unigram_model, skipgram=None, freq_threshold=1000):
    # Pun / alternative word cannot be phrases
    if len(alter_word.split('_')) > 1 or len(pun_word.split('_')) > 1:
        logger.info('FAIL: phrase')
        return False, 'phrase'

    if skipgram and skipgram.vocab.index(get_lemma(pun_word)) == skipgram.vocab.unk():
        logger.info('FAIL: unknown pun word: {}'.format(pun_word))
        return False, 'unk to skipgram'

    return True, None


def main(args):
    ensure_exist(args.outdir, is_dir=True)
    json.dump(vars(args), open(os.path.join(args.outdir, 'config.json'), 'w'))

    unigram_model = UnigramModel(args.word_counts_path, args.oov_prob)
    retriever = Retriever(args.doc_file, path=args.retriever_model, overwrite=args.overwrite_retriever_model)

    if args.system.startswith('rule') or args.system == 'keywords' or args.scorer in ('goodman',):
        skipgram = SkipGram.load_model(args.skipgram_model[0], args.skipgram_model[1], embedding_size=args.skipgram_embed_size, cpu=args.cpu)
    else:
        skipgram = None

    if args.scorer == 'random':
        scorer = RandomScorer()
    elif args.scorer == 'surprisal':
        lm = LMScorer.load_model(args.lm_path)
        scorer = SurprisalScorer(lm, unigram_model, local_window_size=args.local_window_size)
    elif args.scorer == 'goodman':
        scorer = GoodmanScorer(unigram_model, skipgram)

    type_recognizer = TypeRecognizer(threshold=args.type_consistency_threshold)

    if args.system == 'rule':
        generator = RulebasedGenerator(retriever, skipgram, type_recognizer, scorer, dist_to_pun=args.distance_to_pun_word)
    elif args.system == 'rule+neural':
        generator = NeuralCombinerGenerator(retriever, skipgram, type_recognizer, scorer, args.distance_to_pun_word, args)
    elif args.system == 'retrieve':
        generator = RetrieveGenerator(retriever, scorer)
    elif args.system == 'retrieve+swap':
        generator = RetrieveSwapGenerator(retriever, scorer)

    puns = json.load(open(args.pun_words))
    # Uniq
    d = {}
    for e in puns:
        d[e['pun_word']] = e
    puns = d.values()
    # Sorting by quality of pun words
    dmeta = fuzzy.DMetaphone()
    homophone = lambda x, y: float(dmeta(x)[0] == dmeta(y)[0])
    length = lambda x, y: float(len(x) > 2 and len(y) > 2)
    freq = lambda x, y: unigram_model.word_counts.get(x, 0) * unigram_model.word_counts.get(y, 0)
    puns = sorted(puns, key=lambda e: (length(e['pun_word'], e['alter_word']),
                                       homophone(e['pun_word'], e['alter_word']),
                                       freq(e['pun_word'], e['alter_word'])),
                  reverse=True)
    num_success = 0
    processed_examples = []
    for example in puns:
        pun_word, alter_word = example['pun_word'], example['alter_word']
        logger.info('-'*50)
        logger.info('INPUT: alter={} pun={}'.format(alter_word, pun_word))
        logger.info('REFERENCE: {}'.format(' '.join(example['tokens'])))
        logger.info('-'*50)

        feasible, reason = feasible_pun_words(pun_word, alter_word, unigram_model, skipgram=skipgram, freq_threshold=args.pun_freq_threshold)
        if not feasible:
            example['fail'] = reason
            continue

        results = generator.generate(alter_word, pun_word, k=args.num_topic_words, ncands=args.num_candidates, ntemps=args.num_templates)
        example['results'] = results
        if not results:
            continue

        results = [r for r in results if r.get('score') is not None]
        results = sorted(results, key=lambda r: r['score'], reverse=True)
        for r in results[:3]:
            logger.info('{:<8.2f}{}'.format(r['score'], ' '.join(r['output'])))

        processed_examples.append(example)
        num_success += 1
        if args.max_num_examples > 0 and num_success >= args.max_num_examples:
            break

    json.dump(processed_examples, open(os.path.join(args.outdir, 'results.json'), 'w'))


if __name__ == '__main__':
    args = parse_args()
    logging_config(os.path.join(args.outdir, 'generate_pun.log'), console_level=logging.INFO)
    main(args)
